In [3]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

# cnn model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot


#from fl_mnist_implementation_tutorial_utils import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np



from sklearn import model_selection
from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf
import json
import math
import os
import cv2
from PIL import Image
import numpy as np
from keras import layers
from keras.applications import DenseNet201
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm
import tensorflow as tf
from keras import backend as K
from pathlib import Path # help manage f
import gc
from functools import partial
from sklearn import metrics
import itertools
import seaborn as sns
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelBinarizer

In [7]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix='/content/drive/My Drive/Datasets/TimeSeries/UCI HAR Dataset/'):
	# load all train
	#trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
    trainX, trainy = load_dataset_group('train', prefix)
    print(trainX.shape, trainy.shape)
	# load all test
	#testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
    testX, testy = load_dataset_group('test', prefix )
    print(testX.shape, testy.shape)
	# zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
	# one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 10, 32
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
	model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
	model.add(Dropout(0.5))
	model.add(MaxPool1D(pool_size=2))
	model.add(Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(repeats=10):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

# run the experiment
run_experiment()


(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
>#1: 89.888
>#2: 92.026
>#3: 87.547
>#4: 92.195
>#5: 90.126
>#6: 90.058
>#7: 91.008
>#8: 90.635
>#9: 90.499
>#10: 91.924
[89.88802433013916, 92.02578663825989, 87.54665851593018, 92.19545125961304, 90.12554883956909, 90.05768299102783, 91.00780487060547, 90.63454270362854, 90.49881100654602, 91.923987865448]
Accuracy: 90.590% (+/-1.298)


In [ ]:
'''
#----Create numpy array of 'zeros' and 'ones' for labelling benign/malignant images resp.
benign_train_label = np.zeros(len(benign_train))
malign_train_label = np.ones(len(malign_train))
#benign_train_label = (-1)*benign_train_label2

X_data = np.concatenate((benign_train, malign_train), axis = 0)
Y_data = np.concatenate((benign_train_label, malign_train_label), axis = 0)
#X_test = np.concatenate((benign_valid, malign_valid), axis = 0)
#Y_test = np.concatenate((benign_test_label, malign_test_label), axis = 0)

s = np.arange(X_data.shape[0])
np.random.shuffle(s)
X_data = X_data[s] #Re-index
Y_data = Y_data[s]

#binarize the labels
#lb = LabelBinarizer()
#label_list = lb.fit_transform(Y_data)

#split data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                    Y_data,
                                                    test_size=0.1,
                                                    random_state=42)

'''

In [46]:
X_train, y_train, X_test, y_test = load_dataset()
print(X_train.shape)

lb = LabelBinarizer()
labely = lb.fit_transform(y_test)
labelx = lb.fit_transform(y_train)

print(labely)

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)
(7352, 128, 9)
[[0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 ...
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]]


Federated Members (clients) as Data Shards

In [53]:
def create_clients(image_list, label_list, num_clients=2, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as
                data shards - tuple of images and label lists.
        args:
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1

    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]
    print(client_names)
    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))}

#create clients
clients = create_clients(X_train, y_train, num_clients=2, initial='client')

['client_1', 'client_2']


Processing and batching clients’ and test data

In [54]:
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)


#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

#process and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

Creating the Multi Layer Perceptron (MLP) model

In [49]:
from keras.optimizers import SGD
class SimpleMLP:
    @staticmethod
    def build(n_timesteps,n_features, n_outputs):
        #verbose, epochs, batch_size = 0, 10, 32
	      #n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1] =
	      model = Sequential()
	      model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
	      model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
	      model.add(Dropout(0.5))
	      model.add(MaxPool1D(pool_size=2))
	      model.add(Flatten())
	      model.add(Dense(100, activation='relu'))
	      model.add(Dense(n_outputs, activation='softmax'))
	      #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	      # fit network
	      #model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	      # evaluate model
	      #_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	      return model




time_model = evaluate_model(X_train, y_train, X_test, y_test)
time_model.summary()
lr = 0.01
comms_round = 100
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
verbose, epochs, batch_size = 0, 10, 32

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 126, 64)           1792      
                                                                 
 conv1d_1 (Conv1D)           (None, 124, 64)           12352     
                                                                 
 dropout (Dropout)           (None, 124, 64)           0         
                                                                 
 max_pooling1d (MaxPooling1  (None, 62, 64)            0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 3968)              0         
                                                                 
 dense (Dense)               (None, 100)               396900    
                                                        

Model Aggregation (Federated Averaging)

In [50]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad




In [79]:


def test_model(X_test,y_test,  model, comm_round):
    #cce = tf.keras.losses.SparseCategoricalCrossentropy()
    cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test, batch_size=100)
    print("Logits shape:", logits.shape)
    print("y_test shape:", y_test.shape)
    #logits = (model.predict(X_test) > 0.5).astype("float")
    #logits = model.predict(X_test)
    #labels = tf.reshape(y_test, (-1, 6))  # Reshape label_list to match logits shape
    loss = cce(y_test, logits)
    #print(logits)
    #loss = cce(y_test)
    #acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(y_test, axis = 1))
    acc = accuracy_score(tf.argmax(logits, axis=1), y_test)
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

Federated Model Training


In [80]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(128, 9, 6)

global_acc_list = []
global_loss_list = []

#commence global training loop
for comm_round in range(comms_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)

    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(128, 9, 6)
        local_model.compile(loss=loss,
                      optimizer=optimizer,
                      metrics=metrics)

        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=0)

        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        #clear session to free memory after each communication round
        K.clear_session()

    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    #update global model
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, y_test, global_model, comm_round)




30/30 [==============================] - 0s 13ms/step
Logits shape: (2947, 6)
y_test shape: (2947, 6)


ValueError: `labels.shape` must equal `logits.shape` except for the last dimension. Received: labels.shape=(17682,) and logits.shape=(2947, 6)

SGD Vs Federated Averaging

In [ ]:
SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(y_train)).batch(320)
smlp_SGD = SimpleMLP()
SGD_model = smlp_SGD.build(784, 10)

SGD_model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)

# fit the SGD training data to model
_ = SGD_model.fit(SGD_dataset, epochs=100, verbose=0)

#test the SGD global model and print out metrics
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)


In [ ]:
def non_iid_x(image_list, label_list, x=1, num_intraclass_clients=10):
        ''' creates x non_IID clients
        args:
            image_list: python list of images or data points
            label_list: python list of labels
            x: none IID severity, 1 means each client will only have one class of data
            num_intraclass_client: number of sub-client to be created from each none IID class,
            e.g for x=1, we could create 10 further clients by splitting each class into 10

        return - dictionary
            keys - clients's name,
            value - client's non iid 1 data shard (as tuple list of images and labels) '''

        non_iid_x_clients = dict()

        #create unique label list and shuffle
        unique_labels = np.unique(np.array(label_list))
        random.shuffle(unique_labels)

        #create sub label lists based on x
        sub_lab_list = [unique_labels[i:i + x] for i in range(0, len(unique_labels), x)]

        for item in sub_lab_list:
            class_data = [(image, label) for (image, label) in zip(image_list, label_list) if label in item]

            #decouple tuple list into seperate image and label lists
            images, labels = zip(*class_data)

            # create formated client initials
            initial = ''
            for lab in item:
                initial = initial + lab + '_'

            #create num_intraclass_clients clients from the class
            intraclass_clients = create_clients(list(images), list(labels), num_intraclass_clients, initial)

            #append intraclass clients to main clients'dict
            non_iid_x_clients.update(intraclass_clients)

        return non_iid_x_clients